# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd, re
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from datetime import datetime, timedelta
from pyspark.sql import types as T

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [2]:
# Read in the data here
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")
print(df.shape)

(3096313, 28)


In [3]:
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [4]:
# Read the Temperature by city Data
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temp_by_city = pd.read_csv(fname)
print(df_temp_by_city.shape)
df_temp_by_city.head()

(8599212, 7)


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [5]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [11]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [8]:
# Performing cleaning tasks on Immigration data 

# Create dictionary of valid i94port codes
re_search = re.compile(r'\'(.*)\'.*\'(.*)\'')
valid_i94port = {}
with open('i94port_valid.txt') as f:
     for line in f:
         match = re_search.search(line)
         valid_i94port[match[1]]=[match[2]]

def convert_datetime(x):
    try:
        start = datetime(1960, 1, 1)
        return start + timedelta(days=int(x))
    except:
        return None
udf_datetime_from_sas = udf(lambda x: convert_datetime(x), T.DateType())            

def clean_i94_data(file):
    '''
    Input: Path to I94 immigration data file
    
    Output: Spark dataframe of I94 immigration data with valid i94port and converted arrival, departure dates
    
    '''
    
    # Read I94 data into Spark
    df_immigration_data = spark.read.format('com.github.saurfang.sas.spark').load(file)

    # Filter out entries where i94port is invalid
    df_immigration_data = df_immigration_data.filter(df_immigration_data.i94port.isin(list(valid_i94port.keys())))
    # Convert to datetime
    df_immigration_data = df_immigration_data.withColumn("arrival_date", udf_datetime_from_sas("arrdate"))
    df_immigration_data = df_immigration_data.withColumn("departure_date", udf_datetime_from_sas("depdate"))
    df_immigration_data = df_immigration_data.withColumn("Allowed_till", udf_datetime_from_sas("dtaddto"))
    
    return df_immigration_data





# Test cleaning function
immigration_file = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat' 
df_immigration = clean_i94_data(immigration_file)
df_immigration.show()





+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+------------+--------------+------------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|arrival_date|departure_date|Allowed_till|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+------------+--------------+------------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1.897628485E9| null

In [11]:
# Clean temperature data
df_temperature=spark.read.format("csv").option("header", "true").load("../../data2/GlobalLandTemperaturesByCity.csv")

# Filter out entries with NaN average temperature
df_temperature=df_temperature.filter(df_temperature.AverageTemperature != 'NaN')

# Remove duplicate locations
df_temperature=df_temperature.dropDuplicates(['City', 'Country'])

@udf()
def get_i94port(city):
    '''
    Input: City name
    
    Output: Corresponding i94port
    
    '''
    
    for key in valid_i94port:
        if city.lower() in valid_i94port[key][0].lower():
            return key

# Add iport94 code with respect to city name
df_temperature=df_temperature.withColumn("i94port", get_i94port(df_temperature.City))

# Remove entries where iport94 is null
df_temperature=df_temperature.filter(df_temperature.i94port != 'null')

#df_temperature=df_temperature.drop("i94port")

# Show results
df_temperature.show()

+----------+-------------------+-----------------------------+---------+--------------------+--------+---------+-------+
|        dt| AverageTemperature|AverageTemperatureUncertainty|     City|             Country|Latitude|Longitude|i94port|
+----------+-------------------+-----------------------------+---------+--------------------+--------+---------+-------+
|1856-01-01|             26.901|                        1.359|      Ife|             Nigeria|   7.23N|    4.05E|    888|
|1852-07-01|             15.488|                        1.395|    Perth|           Australia|  31.35S|  114.97E|    PER|
|1828-01-01|             -1.977|                        2.551|  Seattle|       United States|  47.42N|  121.97W|    SEA|
|1743-11-01|              2.767|                        1.905| Hamilton|              Canada|  42.59N|   80.73W|    HAM|
|1849-01-01|  7.399999999999999|                        2.699|  Ontario|       United States|  34.56N|  116.76W|    ONT|
|1821-11-01|              2.322|

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [12]:
# Immigration dimension

# Path to I94 immigration data 
immigration_data = '/data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'

# Clean the I94 immigration data and parse as df_immigration_clean dataframe
df_immigration_clean = clean_i94_data(immigration_data)

# Extract needed columns for immigration dimension table
dimm_immigration = df_immigration_clean.select(["i94yr", "i94mon", "i94port", "arrival_date", "i94mode", "departure_date", "i94visa"])

dimm_immigration.show()
# Write immigration dimension table to parquet file
#dimm_immigration.write.parquet("/results/dimm_immigration.parquet")

+------+------+-------+------------+-------+--------------+-------+
| i94yr|i94mon|i94port|arrival_date|i94mode|departure_date|i94visa|
+------+------+-------+------------+-------+--------------+-------+
|2016.0|   4.0|    XXX|  2016-04-29|   null|          null|    2.0|
|2016.0|   4.0|    ATL|  2016-04-07|    1.0|          null|    3.0|
|2016.0|   4.0|    WAS|  2016-04-01|    1.0|    2016-08-25|    2.0|
|2016.0|   4.0|    NYC|  2016-04-01|    1.0|    2016-04-23|    2.0|
|2016.0|   4.0|    NYC|  2016-04-01|    1.0|    2016-04-23|    2.0|
|2016.0|   4.0|    NYC|  2016-04-01|    1.0|    2016-04-11|    1.0|
|2016.0|   4.0|    NYC|  2016-04-01|    1.0|    2016-04-14|    2.0|
|2016.0|   4.0|    NYC|  2016-04-01|    1.0|    2016-04-14|    2.0|
|2016.0|   4.0|    NYC|  2016-04-01|    1.0|    2016-04-09|    2.0|
|2016.0|   4.0|    NYC|  2016-04-01|    1.0|    2016-04-18|    1.0|
|2016.0|   4.0|    NYC|  2016-04-01|    1.0|    2016-08-05|    2.0|
|2016.0|   4.0|    TOR|  2016-04-01|    1.0|    

In [13]:
# Create temporary views of the immigration and temperature data
df_immigration.createOrReplaceTempView("imm_table")
df_temperature.createOrReplaceTempView("temp_table")


In [14]:
# Create temperature table representing temperature by city
dimm_temp = spark.sql('''
SELECT temp_table.AverageTemperature as Avg_temperature,
       temp_table.AverageTemperatureUncertainty as Temp_delta,
       temp_table.City as City
FROM temp_table
''')
dimm_temp.show()
# Write temperature table to parquet files
#dimm_temp.write.parquet("/results/dimm_temp.parquet")

+-------------------+------------------+---------+
|    Avg_temperature|        Temp_delta|     City|
+-------------------+------------------+---------+
|             26.901|             1.359|      Ife|
|             15.488|             1.395|    Perth|
|             -1.977|             2.551|  Seattle|
|              2.767|             1.905| Hamilton|
|  7.399999999999999|             2.699|  Ontario|
|              2.322|             2.375|  Spokane|
| 18.874000000000002|             2.017|Abu Dhabi|
|             25.229|             1.094|    Anaco|
|              9.904|1.4369999999999998|      Ica|
|              9.833|             2.182|  Nogales|
|  8.129999999999999|             2.245|  Atlanta|
|             15.552|             2.305|      Mau|
|              3.264|             1.665|   Newark|
| 18.581000000000003|1.8119999999999998|  Springs|
| 26.055999999999997|1.3769999999999998|      Ise|
|             18.722|             2.302|  Orlando|
|             11.602|2.81600000

In [17]:
# Extract columns for Country table
dimm_city = df_temperature.select(["City", "Country", "Latitude", "Longitude"])

dimm_city.show()
# Write country dimension table to parquet files
#dimm_city.write.parquet("/results/dimm_country.parquet")

+---------+--------------------+--------+---------+
|     City|             Country|Latitude|Longitude|
+---------+--------------------+--------+---------+
|      Ife|             Nigeria|   7.23N|    4.05E|
|    Perth|           Australia|  31.35S|  114.97E|
|  Seattle|       United States|  47.42N|  121.97W|
| Hamilton|              Canada|  42.59N|   80.73W|
|  Ontario|       United States|  34.56N|  116.76W|
|  Spokane|       United States|  47.42N|  117.24W|
|Abu Dhabi|United Arab Emirates|  24.92N|   54.98E|
|    Anaco|           Venezuela|   8.84N|   64.05W|
|      Ica|                Peru|  13.66S|   75.14W|
|  Nogales|       United States|  31.35N|  111.20W|
|  Atlanta|       United States|  34.56N|   83.68W|
|      Mau|               India|  26.52N|   84.18E|
|   Newark|       United States|  40.99N|   74.56W|
|  Springs|        South Africa|  26.52S|   28.66E|
|      Ise|             Nigeria|   7.23N|    5.68E|
|  Orlando|       United States|  28.13N|   80.91W|
|   Laredo| 

In [ ]:
# Create the fact table by joining the immigration and temperature views
fact_table = spark.sql('''
SELECT imm_table.i94bir as Age,
       imm_table.gender as Gender,
       imm_table.biryear as Birth_year,
       imm_table.i94yr as year,
       imm_table.i94mon as month,
       imm_table.i94addr as Address,
       imm_table.i94port as i94port,
       temp_table.City as City,
       imm_table.arrival_date as arrival_date,
       imm_table.departure_date as departure_date,
       imm_table.i94visa as Visa_type
FROM imm_table
JOIN temp_table ON (imm_table.i94port = temp_table.i94port)
''')
fact_table.show()
# Write fact table to parquet files
#fact_table.write.parquet("/results/fact.parquet")

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# quality checks
def data_quality_check(table, desc):
    '''
    Input: Table to check, description
    
    Output: returns 0 if no error and error message if there is
    
    '''
    
    err = df.count()
    if err == 0:
        print("Data quality check failed for {} with zero records".format(desc))
    else:
        print("Data quality check passed for {} with {} records".format(desc, err))
    return 0

# Run Data Quality check function
data_quality_check(df_immigration, "immigration table")
data_quality_check(df_temperature, "temperature table")

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.